In [2]:
import os
import json
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import csv

from collections import Counter, defaultdict

from sklearn.model_selection import train_test_split
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# json 데이터셋 구축

비율 조정 train:dev:test = 8:1:1

In [27]:
##### 새 json 파일로 데이터셋 구축
def getSen(csvfile, desired_age, desired_gender, used_sentences):
    with open(csvfile, 'r') as f:
        reader = csv.DictReader(f)
        rows = list(reader)
        for row in rows:
            if row['age'] == desired_age and row['gender'] == desired_gender:
                sentence = row["sentence"]
                if sentence not in used_sentences:
                    used_sentences.add(sentence)
                    return sentence
        return None

def makeDic(cnt, num, label, filename, SDDfile, age, gender, used_sentences, csvfile, data):
    def get_label_number(label):
        labels = {"기쁨": 0, "슬픔": 1, "분노": 2, "당황": 3, "불안": 4, "상처": 5}
        return labels.get(label, -1)
    if((num < 0) or (cnt < num)):
        dic = {}
        dic['path'] = f"/media/jeeyoung/8C0CACF80CACDE88/JEE/data_multimodal/emotion/한국인감정인식을위한복합영상/Total/{label}/{filename}"
        dic['sentence'] = getSen(SDDfile, age, gender, used_sentences)
        dic['label'] = get_label_number(label)

        if dic['label'] == -1:
            print("label error")
        else:
            data.append(dic)
            with open(csvfile, 'a', newline='') as f:
                wr = csv.writer(f)
                wr.writerow([dic['sentence'], dic['label']])

        cnt += 1  # Increment the count inside the loop
        return data, cnt

    return data, cnt

def construct(mode, label, numList, used_sentences):
    SDDfile = "/media/jeeyoung/8C0CACF80CACDE88/JEE/data_multimodal/emotion/018.감성대화/"+label+"_non.csv"
    KEDfile = "/home/jeeyoung/dataset/"+mode+"_data.json"
    csvfile = "/home/jeeyoung/"+mode+".csv"

    with open(KEDfile, 'r') as f1:
        emotion = json.load(f1)
    
    data = {
    "train": [],
    "val": [],
    "test": []
    }

    count = 0
    cntList = [0,0,0,0,0,0,0]
    for i in range(len(emotion)):
        if(emotion[i]["faceExp_uploader"] == label):
            if((emotion[i]["annot_A"]["faceExp"] == label) and (emotion[i]["annot_B"]["faceExp"] == label) and (emotion[i]["annot_C"]["faceExp"] == label)):
                count += 1
                ###### 청소년 남
                if(emotion[i]["age"] == 10 and emotion[i]["gender"] == "남"):
                    data[mode], cntList[0] = makeDic(cntList[0], numList[0], label, emotion[i]["filename"], SDDfile, "청소년", "남성", used_sentences, csvfile, data[mode])
                ###### 청소년 여
                elif(emotion[i]["age"] == 10 and emotion[i]["gender"] == "여"):
                    data[mode], cntList[1] = makeDic(cntList[1], numList[1], label, emotion[i]["filename"], SDDfile, "청소년", "여성", used_sentences, csvfile, data[mode])
                ###### 청년 남
                elif((emotion[i]["age"] == 20 and emotion[i]["gender"] == "남") or (emotion[i]["age"] == 30 and emotion[i]["gender"] == "남")):
                    data[mode], cntList[2] = makeDic(cntList[2], numList[2], label, emotion[i]["filename"], SDDfile, "청년", "남성", used_sentences, csvfile, data[mode])
                ###### 청년 여
                elif((emotion[i]["age"] == 20 and emotion[i]["gender"] == "여") or (emotion[i]["age"] == 30 and emotion[i]["gender"] == "여")):
                    # 기쁨 20 여 유실된 데이터 제거
                    if (
                        emotion[i]["filename"].find("2aa6b196295318f895d82db48aa037669ef6d7e1a430e300d40dcf47e8712274") == 0
                        and label == "기쁨"
                        and emotion[i]["age"] == 20
                        and emotion[i]["gender"] == "여"
                    ):
                        None
                    else:
                        data[mode], cntList[3] = makeDic(cntList[3], numList[3], label, emotion[i]["filename"], SDDfile, "청년", "여성", used_sentences, csvfile, data[mode])
                ###### 중년 남
                elif((emotion[i]["age"] == 40 and emotion[i]["gender"] == "남") or (emotion[i]["age"] == 50 and emotion[i]["gender"] == "남")):
                    data[mode], cntList[4] = makeDic(cntList[4], numList[4], label, emotion[i]["filename"], SDDfile, "중년", "남성", used_sentences, csvfile, data[mode])
                ###### 중년 여
                elif((emotion[i]["age"] == 40 and emotion[i]["gender"] == "여") or (emotion[i]["age"] == 50 and emotion[i]["gender"] == "여")):
                    data[mode], cntList[5] = makeDic(cntList[5], numList[5], label, emotion[i]["filename"], SDDfile, "중년", "여성", used_sentences, csvfile, data[mode])
                ###### 노년 여
                elif(emotion[i]["age"] == 60 and emotion[i]["gender"] == "여"):
                    data[mode], cntList[6] = makeDic(cntList[6], numList[6], label, emotion[i]["filename"], SDDfile,  "노년", "여성", used_sentences, csvfile, data[mode])
    print(count, label, cntList)
    return data[mode]

def neutralSen(SDDfile, used_sentences):
    with open(SDDfile, 'r') as f:
        reader = csv.DictReader(f)
        rows = list(reader)

    for row in rows:
        sentence = row["sentence"]
        if sentence not in used_sentences:
            used_sentences.add(sentence)
            return sentence

def neutralDic(cnt, num, label, filename, SDDfile, used_sentences, csvfile, data):
    if((num < 0) or (cnt < num)):
        dic = {}
        dic['path'] = f"/media/jeeyoung/8C0CACF80CACDE88/JEE/data_multimodal/emotion/한국인감정인식을위한복합영상/Total/{label}/{filename}"
        dic['sentence'] = neutralSen(SDDfile, used_sentences)
        dic['label'] = 6

        data.append(dic)
        with open(csvfile, 'a', newline='') as f:
            wr = csv.writer(f)
            wr.writerow([dic['sentence'], dic['label']])

    return data, cnt + 1

def neutral(mode, label, numList, used_sentences):
    SDDfile = "/media/jeeyoung/8C0CACF80CACDE88/JEE/data_multimodal/emotion/018.감성대화/"+label+"_non.csv"
    KEDfile = "/home/jeeyoung/dataset/"+mode+"_data.json"
    csvfile = "/home/jeeyoung/"+mode+".csv"
    with open(KEDfile, 'r') as f1:
        emotion = json.load(f1)
    
    data = {
    "train": [],
    "val": [],
    "test": []
    }
    
    count = 0
    cntList = [0,0,0,0,0,0,0]
    for i in range(len(emotion)):
        if(emotion[i]["faceExp_uploader"] == label):
            if((emotion[i]["annot_A"]["faceExp"] == label) and (emotion[i]["annot_B"]["faceExp"] == label) and (emotion[i]["annot_C"]["faceExp"] == label)):
                count += 1
                ###### 청소년 남
                if(emotion[i]["age"] == 10 and emotion[i]["gender"] == "남"):
                    data[mode], cntList[0] = neutralDic(cntList[0], numList[0], label, emotion[i]["filename"], SDDfile, used_sentences, csvfile, data[mode])
                ###### 청소년 여
                elif(emotion[i]["age"] == 10 and emotion[i]["gender"] == "여"):
                    data[mode], cntList[1] = neutralDic(cntList[1], numList[1], label, emotion[i]["filename"], SDDfile, used_sentences, csvfile, data[mode])
                ###### 청년 남
                elif((emotion[i]["age"] == 20 and emotion[i]["gender"] == "남") or (emotion[i]["age"] == 30 and emotion[i]["gender"] == "남")):
                    data[mode], cntList[2] = neutralDic(cntList[2], numList[2], label, emotion[i]["filename"], SDDfile, used_sentences, csvfile, data[mode])
                ###### 청년 여
                elif((emotion[i]["age"] == 20 and emotion[i]["gender"] == "여") or (emotion[i]["age"] == 30 and emotion[i]["gender"] == "여")):
                    data[mode], cntList[3] = neutralDic(cntList[3], numList[3], label, emotion[i]["filename"], SDDfile, used_sentences, csvfile, data[mode])
                ###### 중년 남
                elif((emotion[i]["age"] == 40 and emotion[i]["gender"] == "남") or (emotion[i]["age"] == 50 and emotion[i]["gender"] == "남")):
                    data[mode], cntList[4] = neutralDic(cntList[4], numList[4], label, emotion[i]["filename"], SDDfile, used_sentences, csvfile, data[mode])
                ###### 중년 여
                elif((emotion[i]["age"] == 40 and emotion[i]["gender"] == "여") or (emotion[i]["age"] == 50 and emotion[i]["gender"] == "여")):
                    data[mode], cntList[5] = neutralDic(cntList[5], numList[5], label, emotion[i]["filename"], SDDfile, used_sentences, csvfile, data[mode])
                ###### 노년 여
                elif(emotion[i]["age"] == 60 and emotion[i]["gender"] == "여"):
                    data[mode], cntList[6] = neutralDic(cntList[6], numList[6], label, emotion[i]["filename"], SDDfile, used_sentences, csvfile, data[mode])
    print(label, count)
    return data[mode]

def extendList(object, list):
    if object is not None:
        list.extend(object)
    
def combine(labels, lists, data):
    used_sentences = set()
    data["train"].extend(construct("train", labels, lists[0], used_sentences))
    data["val"].extend(construct("val", labels, lists[1], used_sentences))
    data["test"].extend(construct("test", labels, lists[2], used_sentences))
    print(len(used_sentences))
    return data

def write_csv_header(csvfile):
    with open(csvfile, 'w', newline='') as f:
        wr = csv.writer(f)
        wr.writerow(['sentence', 'label'])

def totalData(label, n0, n1, n2, n3, n4, n5, n6):
    data = {
    "train": [],
    "val": [],
    "test": []
    }

    write_csv_header("/home/jeeyoung/train.csv")
    write_csv_header("/home/jeeyoung/val.csv")
    write_csv_header("/home/jeeyoung/test.csv")

    combine(label[0], n0, data)
    combine(label[1], n1, data)
    combine(label[2], n2, data)
    combine(label[3], n3, data)
    combine(label[4], n4, data)
    combine(label[5], n5, data)
    used_sentences = set()
    data["train"].extend(neutral("train", label[6], n6[0], used_sentences))
    data["val"].extend(neutral("val", label[6], n6[1], used_sentences))
    data["test"].extend(neutral("test", label[6], n6[2], used_sentences))\

    with open("/home/jeeyoung/train.json",'a') as f:
        json.dump(data["train"], f, ensure_ascii=False, indent=4)
    with open("/home/jeeyoung/valid.json",'a') as f:
        json.dump(data["val"], f, ensure_ascii=False, indent=4)
    with open("/home/jeeyoung/test.json",'a') as f:
        json.dump(data["test"], f, ensure_ascii=False, indent=4)

labelList = ["기쁨", "슬픔", "분노", "당황", "불안", "상처", "중립"]

# 청소년 남, 청소년 여, 청년 남, 청년 여, 중년 남, 중년 여, 노년 여 (수 조정X인 경우는 -1)
# 기쁨0
list0 = [[-1,-1,1339,1835,1115,1303,-1],
        [-1,-1,125,319,112,174,-1],
        [-1,-1,123,253,132,167,-1]]
# 슬픔1
list1 = [[-1,-1,1958,2289,1588,1570,-1],
        [-1,-1,193,368,172,297,-1],
        [-1,-1,194,265,202,257,-1]]
# 분노2
list2 = [[-1,-1,1794,2330,1453,1752,-1],
        [-1,-1,178,403,174,337,-1],
        [-1,-1,184,309,158,193,-1]]
# 당황3
list3 = [[-1,-1,1638,2155,1398,1481,-1],
        [-1,-1,123,412,134,307,-1],
        [-1,-1,169,295,172,156,-1]]
# 불안4
list4 = [[-1,-1,1732,2413,-1,-1,-1],
        [-1,-1,146,389,-1,-1,-1],
        [-1,-1,210,294,-1,-1,-1]]
# 상처5
list5 = [[-1,-1,1705,-1,-1,-1,-1],
        [-1,-1,96,-1,-1,-1,-1],
        [-1,-1,196,-1,-1,-1,-1]]
# 중립6
list6 = [[-1,-1,1968,2253,1620,1670,-1],
        [-1,-1,181,383,146,241,-1],
        [-1,-1,196,286,196,213,-1]]

totalData(labelList, list0, list1, list2, list3, list4, list5, list6)

49733 기쁨 [115, 460, 1339, 1835, 1115, 1303, 182]
6571 기쁨 [0, 32, 125, 319, 112, 174, 0]
5857 기쁨 [42, 61, 123, 253, 132, 167, 0]
7889
24298 슬픔 [109, 191, 1958, 2289, 1588, 1570, 130]
3195 슬픔 [0, 18, 193, 368, 172, 297, 0]
2781 슬픔 [34, 40, 194, 265, 202, 257, 0]
9875
26205 분노 [121, 207, 1794, 2330, 1453, 1752, 80]
3583 분노 [0, 7, 178, 403, 174, 337, 0]
3029 분노 [19, 14, 184, 309, 158, 193, 0]
9713
26070 당황 [67, 229, 1638, 2155, 1398, 1481, 24]
3490 당황 [0, 26, 123, 412, 134, 307, 0]
3124 당황 [48, 22, 169, 295, 172, 156, 0]
8856
7755 불안 [63, 124, 1732, 2413, 456, 516, 16]
1097 불안 [0, 3, 146, 389, 56, 201, 0]
882 불안 [8, 2, 210, 294, 53, 20, 0]
6702
4304 상처 [67, 83, 1705, 1844, 280, 304, 9]
562 상처 [0, 5, 96, 253, 39, 168, 0]
418 상처 [1, 4, 196, 163, 46, 7, 0]
5270
중립 39041
중립 5059
중립 4527


In [71]:
###### 인물 겹침 확인
with open('/home/jeeyoung/dataset/val_data.json', 'r') as f:
    emotion_data = json.load(f)

for entry in emotion_data:
    if all(entry[f"annot_{letter}"]["faceExp"] == emotion_label for letter in "ABC"):
        age = entry["age"]
        gender = entry["gender"]
        if(age == 10 and gender == "남"):
            print(entry["filename"])

# 구축 데이터셋 분석

In [2]:
def count_labels(file_path):
    label_counts = [0, 0, 0, 0, 0, 0, 0, 0]
    with open(file_path, 'r') as f:
        emotions = json.load(f)
        for emotion in emotions:
            label = emotion["label"]
            if label >= 0 and label <= 6:
                label_counts[label] += 1
            else:
                label_counts[7] += 1
    return label_counts

train_file_path = '/home/jeeyoung/train.json'
train_label_counts = count_labels(train_file_path)
print(*train_label_counts)

valid_file_path = '/home/jeeyoung/valid.json'
valid_label_counts = count_labels(valid_file_path)
print(*valid_label_counts)

test_file_path = '/home/jeeyoung/test.json'
test_label_counts = count_labels(test_file_path)
print(*test_label_counts)



6660 8089 8022 7614 8006 7965 10367 0
0 120 120 120 120 120 1077 0
213 393 382 393 393 393 1288 0


In [56]:
############### 이미지 파일 존재 여부 파악
import json
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = False

cnt = 0
with open('/home/jeeyoung/valid.json', 'r') as f:
    data = json.load(f)
    for i in range(len(data)):
        file = data[i]["path"]
        if os.path.isfile(file):
            #cnt += 1
            try:
                with Image.open(file) as image:
                    cnt += 1
                    pass
            except OSError as e:
                print(f"Error reading image file: {file}")
                print(f"Error details: {e}")
        else:
            print(f"File {file} does not exist.")
print(cnt)

3696


# misclassified data 분석

In [ ]:
##### VL-KE-T5, KE-T5, ViT의 misclassified 파일 비교 -> 단일모델에서만 잘 구분하지 못한 데이터 추출
import csv
import json
import os

with open('./output/VisionT5MeanBiEncoderForClassification-google_vit-base-patch16-384-KETI-AIR_ke-t5-base/misclassified_samples.csv', 'r') as first_file:
    first_data = list(csv.reader(first_file))

first_image_names = set(row[0] for row in first_data)

csv_directory = '/media/jeeyoung/8C0CACF80CACDE88/JEE/models/Sentiment_analysis/ke-t5/datasets/model_data/error'
second_data = []

for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        csv_file_path = os.path.join(csv_directory, filename)
        with open(csv_file_path, 'r') as file:
            csv_data = list(csv.reader(file))
            second_data.extend(csv_data)

with open('./output/vit/misclassified_samples.csv', 'r') as third_file:
    third_data = list(csv.reader(third_file))

with open('/home/jeeyoung/test.json', 'r') as json_file:
    json_data = json.load(json_file)

def get_json_data(identifier):
    for entry in json_data:
        if identifier in entry.values():
            return entry
    return None

with open('./output/misclassified.csv', 'w', newline='') as output_file:
    csv_writer = csv.writer(output_file)
    csv_writer.writerow(['image name', 'sentence', 'predict label2', 'predict label3', 'true label', 'SECOND/THIRD'])

    for row in second_data:
        sentence = row[0]
        predict_label2 = row[2]

        json_entry = get_json_data(sentence)
        if json_entry:
            image_name = json_entry['path']
            if image_name not in first_image_names:
                true_label = json_entry['label']
                csv_writer.writerow([image_name, sentence, predict_label2, " ", true_label, 'SECOND'])

    for row in third_data:
        image_name = row[0]
        predict_label3 = row[2]

        if image_name not in first_image_names:
            json_entry = get_json_data(image_name)
            if json_entry:
                sentence = json_entry['sentence']
                true_label = json_entry['label']
                csv_writer.writerow([image_name, sentence, " ", predict_label3, true_label, 'THIRD'])

In [2]:
##### KE-T5, ViT 공통으로 분류하지 못한 데이터
import pandas as pd

df = pd.read_csv('/media/jeeyoung/8C0CACF80CACDE88/JEE/models/VL-KE-T5/output/misclassified.csv')

df.iloc[:, 0] = df.iloc[:, 0].str.replace('/media/jeeyoung/8C0CACF80CACDE88/JEE/data_multimodal/emotion/한국인감정인식을위한복합영상/Total/', '')   # 앞 경로 제거
df = df.drop(columns=[df.columns[5]])

def merge_and_clean_rows(duplicate_df):
    group_size = len(duplicate_df)
    if group_size == 1:
        return duplicate_df
    
    merged_row = duplicate_df.iloc[0].copy()
    
    merged_row['predict label2'] = merged_row['predict label2'] if pd.notna(merged_row['predict label2']) else duplicate_df.iloc[1]['predict label2']   \ 
    merged_row['predict label3'] = duplicate_df.iloc[1]['predict label3']
    
    return merged_row

duplicate_field1 = df[df.duplicated('image name', keep=False)]
duplicate_field2 = df[df.duplicated('sentence', keep=False)]

merged_field1 = duplicate_field1.groupby('image name').apply(merge_and_clean_rows)
merged_field2 = duplicate_field2.groupby('sentence').apply(merge_and_clean_rows)

final_merged_result = pd.concat([merged_field1, merged_field2], ignore_index=True)
final_merged_result = final_merged_result.drop_duplicates()
final_merged_result.to_csv('/home/jeeyoung/total_error.csv', index=False)

In [1]:
##### KE-T5, ViT 공통이 아닌 데이터
df = pd.read_csv('/media/jeeyoung/8C0CACF80CACDE88/JEE/models/VL-KE-T5/output/misclassified.csv')
duplicate_rows = df[df.duplicated(['image name', 'sentence'], keep=False)]
deduplicated_df = df.drop(duplicate_rows.index)
deduplicated_df.to_csv('/home/jeeyoung/deduplicated.csv', index=False)